# Miniproyecto 2: Machine Learning con Valor
#### **Curso:** Introducción a Minería de Datos y Machine Learning

---

**¡Felicidades por tu excelente trabajo en el MP1!**

Tu labor en el preprocesamiento de la base de datos ha causado un gran impacto en **AMAZOFF**.

El equipo de Customer Experience ha quedado impresionado con tu habilidad para transformar datos sucios en información
útil y ahora quieren aprovechar tus talentos para enfrentar un nuevo desafío.

Nos han pedido desarrollar un **Predictor de Ratings de Pedidos** utilizando los datos preprocesados de ratings de
pedidos de nuestra plataforma de e-commerce.

# Pauta de Evaluación

Este MP2 está enfocado en responder preguntas de negocio relevantes para **AMAZOFF**.

### Preguntas de Negocio

1. (15 puntos) El equipo necesita extraer la máxima información de los datos. ¿Cómo modificarías la base de datos para ser usada con modelos de ML?
2. (15 puntos) ¡Wow! Gran *feature engineering*. Toca armar tu set para entrenar el modelo. ¿Podrías generar gráficas para el equipo de Customer Experience que expliquen mejor los datos nuevos? ¡No olvides explicarlas!
3. (10 puntos) Dicen que se pueden encontrar patrones en tus datos de entrenamiento. ¿Qué patrones encuentras en los pedidos? ¿Cómo pueden ser de utilidad para mejorar las ventas de AMAZOFF?
4. (25 puntos) ¡Viva el ML! Toca predecir la satisfacción del cliente. ¿Es posible predecir bien el `rating_class`?
5. (20 puntos) El equipo de Customer Experience dice que un solo modelo no es suficiente. ¿Podrías implementar otro?
6. (15 puntos) ¡Dos modelos! Eso es genial. ¿Qué modelo funcionó peor? ¿Si utilizas PCA podría mejor?

### Tener en consideración:

En caso de que el código esté bien, pero no se responda (usando celdas de texto) la pregunta de negocio (sección **Explicación**), **se asignará máximo la mitad de puntos** de esa pregunta.

El miniproyecto está diseñado para completar código. Suba el notebook solo con el código para responder la pregunta de negocio.

> # ¡NO OLVIDES GUARDAR Y SUBIR EL NOTEBOOK A LA PLATAFORMA CUANDO TERMINES! FORMATO .ipynb

# Preámbulo

**¡Cuidado!** En esta sección solo puedes modificar la sección de PÁRAMETROS.

In [1]:
# ¡No modificar esta celda! No está permitido usar librerías adicionales.

# Association Rules
from mlxtend.frequent_patterns import apriori, association_rules

# Machine Learning
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

# General Data Science
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [12]:
# Esta configuración permite que los dataframes se muestren completos.
pd.set_option('display.max_columns', None)

In [15]:
# PARÁMETROS (puedes modificar esta sección)

DATA_PATH = ('C:/Users/nicol/OneDrive/Desktop/ecommerce.csv') # Si tienes el archivo en otro lugar, cámbialo.
RANDOM_STATE = 0 # ¡Cámbialo a tu número favorito!

In [ ]:
# ¡No modificar! Esta celda se encarga de cargar los datos.
df = pd.read_csv(DATA_PATH)
df.purchased = pd.to_datetime(df.purchased)
df.delivered = pd.to_datetime(df.delivered)
df.estimated_delivery = pd.to_datetime(df.estimated_delivery)
print(f'{df.shape[0]} rows x {df.shape[1]} columns')
df.head(5)

In [ ]:
# ¿Cuántos valores únicos hay en cada columna?
for col in df.columns:
    print(f'{col}: {df[col].nunique()} unique values')

# 1. El equipo necesita extraer la máxima información de los datos. ¿Cómo modificarías la base de datos para ser usada con modelos de ML?

⁉️ **Consideración:** La última celda de esta sección es la de cálculo de `rating_class`, mantener el orden. Agregar las celdas para modificar
los datos, antes de esta. No olvides modificar el código para calcular la clase de rating.

ℹ️ **HINT:** Utiliza una celda para cada columna. ¿Qué columnas no sirven? ¿Qué columnas necesito que sean números? ¿Nuevas columnas?

📖 **Referencia Externa:** [Feature Engineering](https://www.freecodecamp.org/news/feature-engineering-techniques-for-structured-data/).

---

✅ **Explicación:**
Para modificar la base de datos para ser usada con modelos de ML, necesitamos extraer los datos relevenates y eliminar los que no son relevantes que puedan afectar al rendimiento del modelo. Ej:
- Eliminar columnas no relevantes
- Convertir columnas de texto a categorías
- Convertir columnas de texto a números
- Crear nuevas columnas con información relevante
- Eliminar valores faltantes o inconsistentes
- Normalizar o escalar los datos
- Crear una nueva columna con la clase de rating basada en la columna de rating.

En un aspecto mas tecnico, es necesario realizar una serie de pasos de feature engineering y preprocesamiento de datos.

In [22]:
# Eliminar columnas irrelevantes
data = df.drop(['order', 'product', 'seller', 'customer', 'city', 'state', 
                  'purchased', 'delivered', 'estimated_delivery'], axis=1)

In [23]:
# Paso 2: Imputación de valores faltantes
# Asegurarse de que no haya valores nulos en 'rating'
data = df.dropna(subset=['rating'])

In [26]:
# Paso 3: Codificación de variables categóricas
label_encoders = {}
for column in ['category', 'payment']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [27]:
# Paso 4: Escalado de características
scaler = StandardScaler()
df[['price', 'shipping']] = scaler.fit_transform(data[['price', 'shipping']])


In [28]:
# Separación en características (X) y variable objetivo (y)
X = data.drop('rating', axis=1)
y = data['rating']

In [ ]:
# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Preparación de datos completada.")

Debido a que estaremos realizando clasificación, pasaremos `rating`de numérico a categórico. Deberás modificar `RATING_BINS`
y `RATING_LABELS` para seleccionar cómo realizar el *binning* de esta clase.

**Ejemplo:**
```python
    RATING_BINS = [0, 2, 3.5, 5]
    RATING_LABELS = ['low', 'medium', 'high']
```

In [ ]:
# Modifica RATING_BINS y RATING_LABELS según consideres.
RATING_BINS = [0, 2, 3, 5]
RATING_LABELS = ['low', 'medium', 'high']

################################################################################

# ¡No modificar! Este código se encarga de crear la columna rating_class.
df['rating_class'] = pd.cut(df['rating'], bins=RATING_BINS, labels=RATING_LABELS, include_lowest=True)
df.value_counts(['rating', 'rating_class'], normalize=True, sort=False, dropna=False)

**¡Ten cuidado!** Debes intentar que tus `rating_class` no sean demasiado imbalancedas. Por ejemplo, que el 90% de los datos
sean *low*, 5% *medium* y 5% *high*. Es preferible algo como: 40% *low*, 30% *medium* y 30% *high*.

In [ ]:
# ¡No modificar!
df.value_counts('rating_class', normalize=True, sort=False, dropna=False)

# 2. ¡Wow! Gran *feature engineering*. Toca armar tu set para entrenar el modelo. ¿Podrías generar gráficas para el equipo de Customer Experience que expliquen mejor los datos nuevos? ¡No olvides explicarlas!

⁉️ **Consideración:** Modifica la lista de columnas a eliminar para el *train*. Debes de hacer por lo menos dos gráficos.

ℹ️ **HINT 1:** ¿Por qué tus gráficas aportan al equipo de Customer Experience? ¡Explica tus nuevas columnas! ¿No tienes? 😓

ℹ️ **HINT 2:** Un buen gráfico tiene título, *labels*, leyenda... ¡y mucho más! 

📖 **Referencia Externa:** [Tutorial de Seaborn](https://www.datacamp.com/tutorial/seaborn-python-tutorial).

---

✅ **Explicación:**

- **Gráfico 1:** El grafico muestra la distribucion de los ratings de los productos. Para el equipo de Customer Experience es importante ya que permite identificar cuantos productos tienen ratings bajos, Medios y altos.

- **Gráfico 2:** Este grafico compara el rating promedio de diferentes categorias de productos y permite ver si hay alguna categoria con ratings bajos.

In [ ]:
# Gráfico 1:  Distribuição de Ratings

########################################################################################################################

plt.figure(figsize=(8, 6))
sns.histplot(data['rating'], bins=10, kde=True)
plt.title('Distribución de Ratings')
plt.xlabel('Rating')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Gráfico 2: Comparacion de Ratings por Categoria de Productos

########################################################################################################################

plt.figure(figsize=(10, 8))
sns.boxplot(data=data, x='category', y='rating')
plt.title('Comparación de Ratings por Categoría de Productos')
plt.xlabel('Categoría')
plt.ylabel('Rating')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Modifica la lista de columnas a eliminar según consideres.

x = df.drop(
    columns=[
        # Agrega las columnas que consideres
        'order', 'product', 'seller', 'customer', 'city', 'state', 'purchased', 'delivered', 'estimated_delivery','rating', 'rating_class'
        
    ]
)

########################################################################################################################

# ¡No modificar! Esta celda se encarga de dividir los datos en conjuntos de entrenamiento, validación y prueba.

y = df['rating_class']

x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)

print(f'Train: {x_train.shape[0]} rows')
print(f'Validation: {x_val.shape[0]} rows')
print(f'Test: {x_test.shape[0]} rows')

display(x_train.head(5))

# 3. Dicen que se pueden encontrar patrones en tus datos de entrenamiento. ¿Qué patrones encuentras en los pedidos? ¿Cómo pueden ser de utilidad para mejorar las ventas de AMAZOFF?

⁉️ **Consideración:** Utiliza reglas de asociación para encontrar reglas útiles para nuevas estrategias de venta. Explica los parámetros y métricas de tu modelo.

ℹ️ **HINT 1:** ¿Qué librería se importó en el inicio? Cuidado, ¡solo datos binarios!

ℹ️ **HINT 2:** De las principales reglas encontradas... ¿cómo implementarías una estrategia de venta/marketing utilizando esta información? 

📖 **Referencia Externa:** [Documentación de mlxtend](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/).

---

✅ **Explicación:**

-Se prepararon los datos convirtiendo las columnas categoricas en variables binarias usando dummies para los valores 0 o 1.
-Se generan los itemsets y se aplica el metodo apriori de mlxtend en el dataframe binario expecificando el soporte minimo en (min_support) para encontrar combinaciones.
-Se generan las reglas de asociacion con association_rules sobre los items frecuentes.

In [ ]:
binary_x_train = pd.get_dummies(x_train, columns=['category', 'price', 'shipping', 'payment'], drop_first=True).astype(bool)
binary_x_train.head()

In [61]:
frequent_itemsets = apriori(binary_x_train, min_support=0.05, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head(10)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules.sort_values(by="confidence", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


# 4. ¡Viva el ML! Toca predecir la satisfacción del cliente. ¿Es posible predecir bien el `rating_class`?

⁉️ **Consideraciones:**

- Elige tres sets de hiperparámetros **diferentes** para los tres modelos de clasificación de *Random Forest*.
- Debes modificar al menos 4 hiperparámetros. No cuentes `random_state`ni `n_jobs`.
- Tus sets de hiperparámetros deben pasar la prueba de *overfitting* y *underfitting*.
- Utiliza los resultados de la validación para elegir tus hiperparámetros, no los del testeo.
- No olvides interpretar tus resultados e intenta relacionarlos a los intereses de **AMAZOFF** si es posible.

ℹ️ **HINT:** El ML es un proceso iterativo, a veces si es que no funcionan bien tus resultados debes de volver a empezar.

📖 **Referencia Externa:** [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

---

✅ **Explicación:**

- **Justifica tus hiperpámetros:** 
-El hiperparametro n_estimators son el numero de valores que permite capturar los patrones sin incrementar demasiado el tiempo de entrenamiento.
-El hiperparametro max_depth es el maximo de profundidad de los arboles, para impedir que se vuelvan mas complejos.
-El hiperparametro min_samples_split es el minimo de muestras que se requieren.
-El hiperparametro max_features selecciona la raiz cuadrada de caracteristicas en cada division.
- **¿Hay *overfitting* o *underfitting*:** Es underfitting ya que la precision y el recall estan relativamente bajos.
- **Resultados de Validación:** 
-Los resultados muestran que el modelo RF1 tiene un rendimiento general de 64%
-RF2 presenta un comportamiento similar, con precision y recall equilibrados para high y low
-RF3 muestra menos precision general que puede sugerir que al modelo le faltan patrones complejos
- **Resultados de Testeo:** Muestra una precision global similar a la validacion
- **¿Es posible predecir bien el `rating_class`:** El modelo es bueno para predecir las clases high y low con un rendimiento aceptable, pero como se vio muestra dificultades para clasificar correctamente la clase medium.

In [62]:
%%time

# Debes de modificar al menos 4 parámetros, sin contar random_state y n_jobs.

rf1 = RandomForestClassifier(
    n_estimators=100,       # Número de árboles en el bosque
    max_depth=10,           # Profundidad máxima de cada árbol
    min_samples_split=5,    # Mínimo de muestras necesarias para dividir un nodo
    max_features='sqrt',    # Número de características a considerar en cada división
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf1.fit(x_train, y_train)

CPU times: total: 5.55 s
Wall time: 712 ms


RandomForestClassifier(max_depth=10, min_samples_split=5, n_jobs=-1,
                       random_state=0)

In [63]:
%%time

# Debes de modificar al menos 4 parámetros, sin contar random_state y n_jobs.

rf2 = RandomForestClassifier(
    n_estimators=200,       # Aumentamos el número de árboles para mejorar la estabilidad
    max_depth=15,           # Mayor profundidad para capturar más detalles
    min_samples_split=10,   # Requiere más muestras para dividir, previniendo sobreajuste
    max_features='log2',    # Cambiamos a log2 para variar el conjunto de características en cada división
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf2.fit(x_train, y_train)

CPU times: total: 16.7 s
Wall time: 1.7 s


RandomForestClassifier(max_depth=15, max_features='log2', min_samples_split=10,
                       n_estimators=200, n_jobs=-1, random_state=0)

In [64]:
%%time

# Debes de modificar al menos 4 parámetros, sin contar random_state y n_jobs.

rf3 = RandomForestClassifier(
    n_estimators=50,             # Menor número de árboles para un modelo más rápido
    max_depth=5,                 # Reducción en la profundidad para simplificar el modelo
    min_samples_leaf=4,          # Mayor número mínimo de muestras por hoja
    min_samples_split=8,         # Divisiones con al menos 8 muestras para evitar sobreajuste
    max_features=None,           # Considera todas las características para reducir variabilidad
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf3.fit(x_train, y_train)

CPU times: total: 2.64 s
Wall time: 373 ms


RandomForestClassifier(max_depth=5, max_features=None, min_samples_leaf=4,
                       min_samples_split=8, n_estimators=50, n_jobs=-1,
                       random_state=0)

In [ ]:
# ¡No modificar! Esta celda se encarga de evaluar los modelos y mostrar los resultados.

y_val_pred1 = rf1.predict(x_val)
y_val_pred2 = rf2.predict(x_val)
y_val_pred3 = rf3.predict(x_val)

print('[Random Forest] Check Overfitting with Accuracy')
print(f'RF1: train={accuracy_score(y_train, rf1.predict(x_train)):.4f} val={accuracy_score(y_val, y_val_pred1):.4f}')
print(f'RF2: train={accuracy_score(y_train, rf2.predict(x_train)):.4f} val={accuracy_score(y_val, y_val_pred2):.4f}')
print(f'RF3: train={accuracy_score(y_train, rf3.predict(x_train)):.4f} val={accuracy_score(y_val, y_val_pred3):.4f}')

In [ ]:
# ¡No modificar! Esta celda se encarga de mostrar los resultados en el set de validación.

print('[Random Forest] Validation Classification Report')
print('#'*80)
print('RF1')
print(classification_report(y_val, y_val_pred1))
print('#'*80)
print('RF2')
print(classification_report(y_val, y_val_pred2))
print('#'*80)
print('RF3')
print(classification_report(y_val, y_val_pred3))

In [ ]:
# ¡Completa el código! Esta celda se encarga de evaluar los modelos en el set de testeo y mostrar los resultados.

y_pred_rf1 = rf1.predict(x_test)
y_pred_rf2 = rf2.predict(x_test)
y_pred_rf3 = rf3.predict(x_test)

print('[Random Forest] Test Classification Report')
print('#'*80)
print('RF1')
print(classification_report(y_test, y_pred_rf1))
print('#'*80)
print('RF2')
print(classification_report(y_test, y_pred_rf2))
print('#'*80)
print('RF3')
print(classification_report(y_test, y_pred_rf3))

# 5. El equipo de Customer Experience dice que un solo modelo no es suficiente. ¿Podrías implementar otro?

⁉️ **Consideración:** Utiliza otro modelo visto en clase, como *KNN* y repite todos los pasos de la pregunta 4. No utilices *Random Forest*.

ℹ️ **HINT 1:** No olvides elegir 3 set de hiperparámetros y analizar los resultados de validación y testeo.

ℹ️ **HINT 2:** Si utilizas un modelo no visto en clases, debes de explicar su funcionamiento para utilizarlo. 

📖 **Referencia Externa:** [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

---

✅ **Explicación:**

- **Justifica tus hiperpámetros:** los hiperparametros elegidos para cada modelo KNN estan ajustados para optimizar la precision y adaptarse a las caracteristicas
- **¿Hay *overfitting* o *underfitting*:** La precision en los conjuntos de entrenamiento y validacion es similar, lo que sugiere que no hay overfitting.
- **Resultados de Validación:** En el conjunto de validacion, los modelos muestran una precision promedio de alrededor del 60%.
- **Resultados de Testeo:** Los resultados en el conjunto de testeo indican que los modelos generalizan bien
- **¿Es posible predecir bien el `rating_class`:** Aunque los modelos pueden predecir razonablemente bien las clases high y low, la baja precision en la clase medium indica que el modelo no es completamente fiable para predecir rating_class

In [68]:
# Instancia tu modelo 1 aquí.

knn1 = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='minkowski')
knn1.fit(x_train, y_train)


KNeighborsClassifier()

In [69]:
# Instancia tu modelo 2 aquí.

knn2 = KNeighborsClassifier(n_neighbors=10, weights='distance', metric='minkowski')
knn2.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [70]:
# Instancia tu modelo 3 aquí.

knn3 = KNeighborsClassifier(n_neighbors=7, weights='distance', metric='manhattan')
knn3.fit(x_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=7, weights='distance')

In [71]:
# Revisa si hay overfitting (resultado de entrenamiento muy superior al de validación).
# Utiliza: `accuracy_score`.

# Calcula la precisión en el conjunto de entrenamiento y validación para cada modelo
train_acc_knn1 = accuracy_score(y_train, knn1.predict(x_train))
val_acc_knn1 = accuracy_score(y_val, knn1.predict(x_val))

train_acc_knn2 = accuracy_score(y_train, knn2.predict(x_train))
val_acc_knn2 = accuracy_score(y_val, knn2.predict(x_val))

train_acc_knn3 = accuracy_score(y_train, knn3.predict(x_train))
val_acc_knn3 = accuracy_score(y_val, knn3.predict(x_val))

# Imprime los resultados para observar posibles signos de overfitting
print(f'KNN1 - Entrenamiento: {train_acc_knn1:.4f}, Validación: {val_acc_knn1:.4f}')
print(f'KNN2 - Entrenamiento: {train_acc_knn2:.4f}, Validación: {val_acc_knn2:.4f}')
print(f'KNN3 - Entrenamiento: {train_acc_knn3:.4f}, Validación: {val_acc_knn3:.4f}')

KNN1 - Entrenamiento: 0.7176, Validación: 0.5970
KNN2 - Entrenamiento: 0.9265, Validación: 0.5962
KNN3 - Entrenamiento: 0.9264, Validación: 0.5948


In [ ]:
# Imprime los resultados de tus modelos en el set de validacion (classification_report).

# Genera las predicciones en el conjunto de validación para cada modelo
y_val_pred_knn1 = knn1.predict(x_val)
y_val_pred_knn2 = knn2.predict(x_val)
y_val_pred_knn3 = knn3.predict(x_val)

# Imprime el reporte de clasificacion para cada modelo
print('[KNN Validation Classification Report]')
print('#'*80)
print('KNN1')
print(classification_report(y_val, y_val_pred_knn1))
print('#'*80)
print('KNN2')
print(classification_report(y_val, y_val_pred_knn2))
print('#'*80)
print('KNN3')
print(classification_report(y_val, y_val_pred_knn3))

In [ ]:
# Imprime los resultados de tus modelos en el set de testeo (classification_report).

# Genera las predicciones en el conjunto de testeo para cada modelo
y_test_pred_knn1 = knn1.predict(x_test)
y_test_pred_knn2 = knn2.predict(x_test)
y_test_pred_knn3 = knn3.predict(x_test)

# Imprime el reporte de clasificacion para cada modelo en el conjunto de testeo
print('[KNN Test Classification Report]')
print('#'*80)
print('KNN1')
print(classification_report(y_test, y_test_pred_knn1))
print('#'*80)
print('KNN2')
print(classification_report(y_test, y_test_pred_knn2))
print('#'*80)
print('KNN3')
print(classification_report(y_test, y_test_pred_knn3))

# 6. ¡Dos modelos! Eso es genial. ¿Qué modelo funcionó peor? ¿Si utilizas PCA podría mejor?

⁉️ **Consideración:** Elige la combinación de modelo e hiperparámetros que peor se desempeño entre las preguntas 4 y 5. Aplica PCA con un número de componentes adecuado y corre los resultados otra vez.

ℹ️ **HINT:** ¿Cómo se elige el número de componentes? ¿Y ese gráfico para qué sirve?

📖 **Referencia Externa:** [PCA](https://www.baeldung.com/cs/pca).

---

✅ **Explicación:**

- **¿Por qué elegiste ese número de componentes?** Se seleccionaron componentes que capturen la mayor varianza de los datos sin perder demasiada informacion.
- **Resultados de Validación:** La precision en la validacion fue baja para la clase "Medium", indicando problemas con las categorias al diferenciarlas
- **Resultados de Testeo:** Similar al set de validacion, pero con mejores resultados en "Low"
- **¿Mejoraron los resultados al usar PCA? ¿Por qué?** PCA no mostro mejoras significativas. Significa que la reduccion de dimensionalidad no fue suficiente

In [ ]:
# ¡No modificar! Gráfico de Varianza Explicada por Número de Componentes

explained_variance = []
for i in range(1, 8):
    pca = PCA(n_components=i)
    pca.fit(x_train)
    explained_variance.append(pca.explained_variance_ratio_.sum())

plt.plot(range(1, 8), explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.title('PCA Explained Variance')
plt.show()

In [75]:
# Elige el número de componentes que consideres adecuado.

pca = PCA(n_components=0.95)

########################################################################################################################

# ¡No modificar! Esta celda se encarga de transformar los datos con PCA.

pca.fit(x_train)

x_train_pca = pca.transform(x_train)
x_val_pca = pca.transform(x_val)
x_test_pca = pca.transform(x_test)

In [ ]:
# Elige el modelo que deseas utilizar. Selecciona los hiperparámetros que consideres. ¿Elegiste la peor combinación anterior?
#     Ejemplo 1: model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE).
#     Ejemplo 2: model = KNeighborsClassifier(n_neighbors=10).

model = KNeighborsClassifier(n_neighbors=5)

########################################################################################################################

# ¡No modificar! Este código se encarga de entrenar el modelo y mostrar los resultados en el set de validación.

model.fit(x_train_pca, y_train)

y_val_pca_pred = model.predict(x_val_pca)

print(f'[{model.__class__.__name__}] Validation Classification Report')
print(classification_report(y_val, y_val_pca_pred))

In [ ]:
# ¡No modificar! Esta celda se encarga de mostrar los resultados en el set de test.

y_test_pca_pred = model.predict(x_test_pca)

print(f'[{model.__class__.__name__}] Test Classification Report')
print(classification_report(y_test, y_test_pca_pred))

> # ¡NO OLVIDES GUARDAR Y SUBIR EL NOTEBOOK A LA PLATAFORMA CUANDO TERMINES! FORMATO .ipynb